# Create crossval tables

This notebook is dedicated to create the cv table to v6 tuning produced during 2017 (commissioning phase) for tracking purpose.


In [1]:
from kolmov import crossval_table, get_color_fader
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

Welcome to JupyROOT 6.23/01
Using all sub packages with ROOT dependence


In [2]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    }
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

In [3]:
etbins = [0, 3, 10, 15]
etabins = [0.0, 0.8, 1.37, 1.54, 2.50]

## 1) Reading all tunings:


In [4]:
cv_v1  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [5]:
# just for comparison
cv_v1.fill(  '/Volumes/castor/tuning_data/Jpsi/v1/r0/*/*/*.gz', 'v1')

2021-02-27 14:54:32,123 | Py.crossval_table                       INFO Reading file for v1 tag from /Volumes/castor/tuning_data/Jpsi/v1/r0/*/*/*.gz
2021-02-27 14:54:32,124 | Py.crossval_table                       INFO There are 1500 files for this task...
2021-02-27 14:54:32,124 | Py.crossval_table                       INFO Filling the table... 
2021-02-27 14:55:07,244 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


In [6]:
cv_v1.to_csv('table_v1.csv')

**NOTE** Use the cell below to rerun all tables from the csv file.

In [7]:
cv_v1.from_csv('table_v1.csv')

### 1.1) Get best inits and sorts:

In [8]:
best_inits_v1 = cv_v1.filter_inits("max_sp_val")

In [9]:
best_sorts_v1 = cv_v1.filter_sorts( best_inits_v1.loc[best_inits_v1.model_idx==0] , 'max_sp_op')

In [10]:
best_inits = pd.concat([
                        best_inits_v1,
                       ])
best_sorts = pd.concat([
                        best_sorts_v1,
                       ])

### 1.2) Make the table:

In [11]:
for op in ['tight','medium','loose','vloose']:
    cv_v1.dump_beamer_table( best_inits ,  [op], 'tuning_v1_'+op, 
                             title = op+' Tunings (v1)', 
                             tags = ['v1']
                           )

2021-02-27 14:55:11,100 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_tight.pdf latex code...
2021-02-27 14:55:18,599 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_medium.pdf latex code...
2021-02-27 14:55:25,325 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_loose.pdf latex code...
2021-02-27 14:55:30,604 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_vloose.pdf latex code...


### 1.3) Plot training curves:

In [12]:
cv_v1.plot_training_curves( best_inits_v1, best_sorts_v1 , 'v1' )